# Задание_1. Практические задания SQL
# Для решения использовать тренажер https://www.w3schools.com/sql/trysql.asp?filename=trysql_select_all

1. Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес.

In [ ]:
SELECT CustomerName, Country, Address
FROM Customers
WHERE Country IN ('Germany', 'France') OR (Country = 'Spain' AND City = 'Madrid')
ORDER BY Country;

Результаты

Number of Records: 25
CustomerName	Country	Address
Blondel père et fils	France	24, place Kléber
Bon app'	France	12, rue des Bouchers
Du monde entier	France	67, rue des Cinquante Otages
Folies gourmandes	France	184, chaussée de Tournai
France restauration	France	54, rue Royale
La corne d'abondance	France	67, avenue de l'Europe
La maison d'Asie	France	1 rue Alsace-Lorraine
Paris spécialités	France	265, boulevard Charonne
Spécialités du monde	France	25, rue Lauriston
Victuailles en stock	France	2, rue du Commerce
Vins et alcools Chevalier	France	59 rue de l'Abbaye
Alfreds Futterkiste	Germany	Obere Str. 57
Blauer See Delikatessen	Germany	Forsterstr. 57
Drachenblut Delikatessend	Germany	Walserweg 21
Frankenversand	Germany	Berliner Platz 43
Königlich Essen	Germany	Maubelstr. 90
Lehmanns Marktstand	Germany	Magazinweg 7
Morgenstern Gesundkost	Germany	Heerstr. 22
Ottilies Käseladen	Germany	Mehrheimerstr. 369
QUICK-Stop	Germany	Taucherstraße 10
Toms Spezialitäten	Germany	Luisenstr. 48
Die Wandernde Kuh	Germany	Adenauerallee 900
Bólido Comidas preparadas	Spain	C/ Araquil, 67
FISSA Fabrica Inter. Salchichas S.A.	Spain	C/ Moralzarzal, 86
Romero y tomillo	Spain	Gran Vía, 1

2. Выберите топ 3 страны по количеству заказчиков, выведите их названия и количество записей.

In [ ]:
SELECT c.Country, COUNT(*) AS Quantity
FROM Customers AS c
GROUP BY c.Country
ORDER BY Quantity DESC
LIMIT 3;

Результаты

Number of Records: 3
Country	Quantity
USA	13
Germany	11
France	11


3. Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления.

In [ ]:
SELECT s.ShipperName, o.OrderDate
FROM (
    SELECT *
    FROM (
        SELECT *
        FROM Orders
        ORDER BY OrderDate ASC
        LIMIT 10
    ) o
    ORDER BY OrderDate DESC
    LIMIT 1
) o
JOIN Shippers s ON o.ShipperID = s.ShipperID;

Результаты

Number of Records: 1
ShipperName	OrderDate
Federal Shipping	1996-07-16

4. Выберите самый дорогой заказ, выведите список товаров с их ценами.

In [ ]:
SELECT p.ProductName, p.Unit, od.Quantity, p.Price
FROM Orders o
JOIN OrderDetails od ON o.OrderID = od.OrderID
JOIN Products p ON od.ProductID = p.ProductID
WHERE o.OrderID = (
    SELECT OrderID
    FROM OrderDetails
    JOIN Products ON OrderDetails.ProductID = Products.ProductID
    GROUP BY OrderID
    ORDER BY SUM(OrderDetails.Quantity * Products.Price) DESC
    LIMIT 1
);

Результаты

Number of Records: 4
ProductName	Unit	Quantity	Price
Sir Rodney's Marmalade	30 gift boxes	12	81
Côte de Blaye	12 - 75 cl bottles	40	263.5
Camembert Pierrot	15 - 300 g rounds	70	34
Mozzarella di Giovanni	24 - 200 g pkgs.	42	34.8

5. Какой товар больше всего заказывали по количеству единиц товара, выведите его название и количество единиц в каждом из заказов.

In [ ]:
SELECT p.ProductName, od.Quantity, od.OrderID
FROM Products p
JOIN (
    SELECT od.ProductID AS pid
    FROM Orders o
    JOIN OrderDetails od ON o.OrderID = od.OrderID
    GROUP BY od.ProductID
    ORDER BY SUM(od.Quantity) DESC
    LIMIT 1
) h ON p.ProductID = h.pid
JOIN OrderDetails od ON p.ProductID = od.ProductID
ORDER BY od.Quantity DESC;

Результаты

Number of Records: 14
ProductName	Quantity	OrderID
Gorgonzola Telino	70	10359
Gorgonzola Telino	60	10390
Gorgonzola Telino	56	10342
Gorgonzola Telino	42	10376
Gorgonzola Telino	40	10272
Gorgonzola Telino	32	10393
Gorgonzola Telino	30	10356
Gorgonzola Telino	30	10374
Gorgonzola Telino	25	10335
Gorgonzola Telino	20	10253
Gorgonzola Telino	20	10363
Gorgonzola Telino	15	10273
Gorgonzola Telino	14	10423
Gorgonzola Telino	4	10325

6. Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон.

In [ ]:
SELECT sup.SupplierName, sup.Country, sup.ContactName, sup.Phone
FROM Suppliers sup
JOIN Products p ON sup.SupplierID = p.SupplierID
JOIN OrderDetails od ON p.ProductID = od.ProductID
JOIN Orders o ON od.OrderID = o.OrderID
GROUP BY sup.SupplierID
ORDER BY COUNT(o.OrderID) DESC
LIMIT 5;

Результаты

Number of Records: 5
SupplierName	Country	ContactName	Phone
Plutzer Lebensmittelgroßmärkte AG	Germany	Martin Bein	(069) 992755
Pavlova, Ltd.	Australia	Ian Devling	(03) 444-2343
Specialty Biscuits, Ltd.	UK	Peter Wilson	(161) 555-4448
Norske Meierier	Norway	Beate Vileid	(0)2-953010
Formaggi Fortini s.r.l.	Italy	Elio Rossi	(0544) 60323

7. Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.

In [ ]:
SELECT cust.Country, cat.CategoryName, SUM(od.Quantity * p.Price) AS TotalPrice
FROM Orders o
JOIN Customers cust ON o.CustomerID = cust.CustomerID
JOIN OrderDetails od ON o.OrderID = od.OrderID
JOIN Products p ON od.ProductID = p.ProductID
JOIN Categories cat ON p.CategoryID = cat.CategoryID
WHERE cust.Country = 'Brazil'
GROUP BY p.CategoryID
ORDER BY TotalPrice DESC
LIMIT 1;

Результаты

Number of Records: 1
Country	CategoryName	TotalPrice
Brazil	Beverages	13690

8. Какая разница в стоимости между самым дорогим и самым дешевым заказом из США.

In [ ]:
SELECT (MAX(orderPrices.OrderPrice) - MIN(orderPrices.OrderPrice)) AS PriceDifference
FROM (
    SELECT o.OrderID, SUM(od.Quantity * p.Price) AS OrderPrice
    FROM Orders o
    JOIN OrderDetails od ON o.OrderID = od.OrderID
    JOIN Products p ON p.ProductID = od.ProductID
    JOIN Customers cust ON cust.CustomerID = o.CustomerID
    WHERE cust.Country = 'USA'
    GROUP BY o.OrderID
) orderPrices;

Результаты

Number of Records: 1
PriceDifference
7638.45

9. Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке.

In [ ]:
SELECT COUNT(*) AS OrdersCount, (e.FirstName || ' ' || e.LastName) AS FullName
FROM Orders o
JOIN Employees e ON o.EmployeeID = e.EmployeeID
GROUP BY e.EmployeeID, FullName
ORDER BY e.BirthDate DESC
LIMIT 3;

Результаты

Number of Records: 3
OrdersCount	FullName
6	Anne Dodsworth
29	Nancy Davolio
31	Janet Leverling

10. Сколько банок крабового мяса всего было заказано.

In [ ]:
SELECT SUM(od.Quantity) AS TotalCrabMeatQuantity
FROM Orders o
JOIN OrderDetails od ON o.OrderID = od.OrderID
JOIN Products p ON od.ProductID = p.ProductID
WHERE p.ProductName = 'Boston Crab Meat';

Результаты

Number of Records: 1
TotalCrabMeatQuantity
256